# Exploratory Data Analysis (EDA) of Processed Sentinel-2 Satellite Imagery

This notebook performs an initial EDA on processed Sentinel-2 satellite imagery for a selected test AOI. The goal is to visually inspect the data, generate various band composites and spectral indices, and identify potential anomalies or features indicative of past human activity or interesting environmental patterns.

In [ ]:
import configparser
from pathlib import Path
import rasterio
from rasterio.plot import show, show_hist
import matplotlib.pyplot as plt
import numpy as np
import rioxarray # For easier band management and calculations with xarray
import xarray as xr # Though rioxarray might handle most direct needs
import geopandas

# Helper function for plotting with colorbar
def plot_raster(data_array, ax, title, cmap='viridis', cbar_label=None):
    im = ax.imshow(data_array, cmap=cmap)
    ax.set_title(title)
    ax.set_xlabel("Easting (pixels)")
    ax.set_ylabel("Northing (pixels)")
    plt.colorbar(im, ax=ax, label=cbar_label, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])

## 1. Configuration and Setup

In [ ]:
CONFIG_FILE_PATH = "../scripts/satellite_pipeline/config.ini" # Adjust if your config is elsewhere
SCRIPT_DIR = Path(".").resolve().parent # Assuming notebook is in 'notebooks' dir, so parent is project root
EDA_OUTPUT_DIR = SCRIPT_DIR / "eda_outputs" / "satellite"
EDA_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def load_config(config_path):
    config = configparser.ConfigParser(interpolation=None, allow_no_value=True)
    if not Path(config_path).exists():
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    config.read(config_path)
    return config

config = load_config(CONFIG_FILE_PATH)

# Get relevant paths from config
base_processed_dir_raw = config['DEFAULT'].get('base_processed_data_dir', '../../data')
s2_processed_suffix = config['DEFAULT'].get('s2_processed_suffix', 'sentinel2/processed') # From original satellite pipeline config

# Construct absolute path for processed_satellite_dir from SCRIPT_DIR (project root)
PROCESSED_SATELLITE_DIR = (SCRIPT_DIR / base_processed_dir_raw.replace('../../', '') / s2_processed_suffix).resolve()

print(f"Processed Satellite Directory: {PROCESSED_SATELLITE_DIR}")
print(f"EDA Output Directory: {EDA_OUTPUT_DIR}")

# AOI definition (example: using the bbox from config for context)
aoi_bbox_str = config['DEFAULT'].get('aoi_bbox')
aoi_geojson_path_str = config['DEFAULT'].get('aoi_geojson_path')
aoi_geom = None

if aoi_geojson_path_str and Path(SCRIPT_DIR / aoi_geojson_path_str.replace('../../','').replace('../','')).exists():
    aoi_geojson_path = Path(SCRIPT_DIR / aoi_geojson_path_str.replace('../../','').replace('../',''))
    aoi_gdf = geopandas.read_file(aoi_geojson_path)
    aoi_geom = aoi_gdf.geometry.iloc[0]
    print(f"Using AOI from GeoJSON: {aoi_geojson_path}")
elif aoi_bbox_str:
    coords = [float(c.strip()) for c in aoi_bbox_str.split(',')]
    minx, miny, maxx, maxy = coords
    aoi_geom = geopandas.GeoSeries([box(minx, miny, maxx, maxy)], crs="EPSG:4326")
    print(f"Using AOI from BBOX (EPSG:4326): {coords}")
else:
    print("No AOI geometry found in config (aoi_geojson_path or aoi_bbox).")

## 2. Load Processed Sentinel-2 Data

We need to find a processed Sentinel-2 GeoTIFF file. These files are typically multi-band, containing the selected bands (e.g., Blue, Green, Red, NIR) as specified in the `config.ini` during the `preprocess_sentinel2.py` step. The filename usually indicates the original product name and processing details.

In [ ]:
# Find a processed Sentinel-2 file
# Example filename: S2A_MSIL2A_20230716T142721_N0509_R025_T20NKE_20230716T203922_Processed_B02B03B04B08_10m.tif
processed_s2_files = list(PROCESSED_SATELLITE_DIR.glob("*_Processed_*.tif"))

if not processed_s2_files:
    raise FileNotFoundError(f"No processed Sentinel-2 files found in {PROCESSED_SATELLITE_DIR} matching '*_Processed_*.tif'")

selected_s2_path = processed_s2_files[0] # Pick the first one for this EDA
print(f"Selected Processed Sentinel-2 File: {selected_s2_path}")

# Load the multi-band raster using rioxarray
# This loads it as an xarray.DataArray, which is convenient for band operations
try:
    s2_data_xr = rioxarray.open_rasterio(selected_s2_path, masked=True)
except rasterio.errors.RasterioIOError as e:
    print(f"Error opening raster file {selected_s2_path}: {e}")
    print("This might be due to the file being empty, corrupt, or GDAL drivers not being available.")
    raise

# Infer band names from config or assume order if not available in metadata
# The preprocess_sentinel2.py script saves bands in the order specified in 'output_bands' config
output_bands_str = config['PREPROCESSING'].get('output_bands', 'B02,B03,B04,B08')
configured_bands = [b.strip().upper() for b in output_bands_str.split(',')]

if len(configured_bands) == s2_data_xr.shape[0]: # Number of bands in file matches config
    s2_data_xr = s2_data_xr.assign_coords(band=configured_bands)
    print(f"Assigned band names from config: {configured_bands}")
else:
    print(f"Warning: Number of bands in file ({s2_data_xr.shape[0]}) does not match 'output_bands' in config ({len(configured_bands)}).")
    print("Band names will be integers. Composites and indices might be incorrect if order is not as expected (B,G,R,NIR,...).")
    # Default band names if they can't be inferred reliably for dictionary access
    # This assumes a default order like B, G, R, NIR for the first 4 bands if names are integers
    # For robust dictionary access, we ensure band coordinates are strings
    s2_data_xr = s2_data_xr.assign_coords(band=[str(b+1) for b in range(s2_data_xr.shape[0])])

print("\nSentinel-2 DataArray properties:")
print(s2_data_xr)

# For plotting, select a single time slice if there's a time dimension (usually not for these processed files)
if 'time' in s2_data_xr.dims:
    s2_data_xr = s2_data_xr.isel(time=0)

# Define common bands for easier access - adjust based on your actual band names/order
# These try to use the configured names first, then fall back to index if names are just numbers
def get_band(data_array, desired_band_name, fallback_index):
    try:
        return data_array.sel(band=desired_band_name)
    except KeyError:
        print(f"Band '{desired_band_name}' not found by name. Trying index {fallback_index}.")
        if fallback_index < len(data_array.band):
            return data_array.isel(band=fallback_index)
        else:
            raise ValueError(f"Band {desired_band_name} (index {fallback_index}) not available in data with bands: {data_array.band.values}")

BLUE = get_band(s2_data_xr, 'B02', 0) # Sentinel-2 Blue: Band 2
GREEN = get_band(s2_data_xr, 'B03', 1) # Sentinel-2 Green: Band 3
RED = get_band(s2_data_xr, 'B04', 2)   # Sentinel-2 Red: Band 4
NIR = get_band(s2_data_xr, 'B08', 3)    # Sentinel-2 NIR: Band 8
# SWIR1 = get_band(s2_data_xr, 'B11', 4) # Sentinel-2 SWIR1: Band 11 (if included)
# SWIR2 = get_band(s2_data_xr, 'B12', 5) # Sentinel-2 SWIR2: Band 12 (if included)

## 3. Band Composites

### 3.1. True Color Composite (Red, Green, Blue)

In [ ]:
# Normalize bands for display (simple percentile clip)
def normalize_for_display(band_array):
    # Ensure it's a numpy array for percentile calculation if it's an xarray.DataArray
    data = band_array.data.astype(np.float32)
    # Handle potential NaNs if data is masked
    if np.isnan(data).any():
        min_val, max_val = np.nanpercentile(data, [2, 98])
    else:
        min_val, max_val = np.percentile(data, [2, 98])
    
    normalized = (data - min_val) / (max_val - min_val)
    return np.clip(normalized, 0, 1)

r_norm = normalize_for_display(RED)
g_norm = normalize_for_display(GREEN)
b_norm = normalize_for_display(BLUE)

true_color_composite = np.dstack((r_norm, g_norm, b_norm))

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(true_color_composite)
ax.set_title('True Color Composite (RGB)')
ax.set_xlabel("Easting (pixels)")
ax.set_ylabel("Northing (pixels)")
ax.set_xticks([])
ax.set_yticks([])
plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_true_color.png")
plt.show()

**Observations (True Color):**
- *[TODO: Add observations. Describe the general appearance. Are there visible signs of modern agriculture, deforestation, settlements, roads, rivers? Any unusual soil colors or patterns?]*

### 3.2. False Color Composite (NIR, Red, Green)

In [ ]:
nir_norm = normalize_for_display(NIR)
# r_norm, g_norm already calculated

false_color_composite_veg = np.dstack((nir_norm, r_norm, g_norm))

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(false_color_composite_veg)
ax.set_title('False Color Composite (NIR-Red-Green - Vegetation Emphasis)')
ax.set_xlabel("Easting (pixels)")
ax.set_ylabel("Northing (pixels)")
ax.set_xticks([])
ax.set_yticks([])
plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_false_color_veg.png")
plt.show()

**Observations (False Color - Vegetation):**
- *[TODO: Add observations. Healthy vegetation should appear bright red. How does this composite highlight different types of vegetation or land cover? Are there geometric patterns in the red tones that might suggest past agriculture or earthworks now overgrown? Any areas of stressed vegetation (less red)?]*

## 4. Spectral Indices

### 4.1. NDVI (Normalized Difference Vegetation Index)

In [ ]:
# Ensure bands are float for calculation
nir_float = NIR.astype(np.float32)
red_float = RED.astype(np.float32)

# Calculate NDVI, handling potential division by zero
ndvi = xr.where(nir_float + red_float == 0, 0, (nir_float - red_float) / (nir_float + red_float))

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_raster(ndvi, ax, title='NDVI (Normalized Difference Vegetation Index)', cmap='RdYlGn', cbar_label='NDVI Value')
plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_ndvi.png")
plt.show()

# Histogram of NDVI values
fig_hist, ax_hist = plt.subplots(1,1, figsize=(8,5))
ndvi.plot.hist(ax=ax_hist, bins=100)
ax_hist.set_title('NDVI Value Distribution')
plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_ndvi_histogram.png")
plt.show()

**Observations (NDVI):**
- *[TODO: Add observations. Describe the NDVI patterns. High values indicate healthy vegetation, low values bare soil or water. Are there any geometric shapes, linear features, or unusually shaped patches of high/low NDVI that don't correspond to modern features visible in the true/false color images? Consider if any patterns might suggest ancient fields, earthworks altering vegetation, or 'terra preta' soils.]*

### 4.2. NDWI (Normalized Difference Water Index)

In [ ]:
# Using Green and NIR: (Green - NIR) / (Green + NIR)
green_float = GREEN.astype(np.float32)
# nir_float already defined

ndwi = xr.where(green_float + nir_float == 0, 0, (green_float - nir_float) / (green_float + nir_float))

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_raster(ndwi, ax, title='NDWI (Normalized Difference Water Index - Green/NIR)', cmap='Blues', cbar_label='NDWI Value')
plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_ndwi.png")
plt.show()

**Observations (NDWI):**
- *[TODO: Add observations. This index highlights open water (high positive values) and can indicate soil/vegetation moisture. Are there any old river channels (paleochannels), potential ancient canals, moats, or reservoirs visible? Are there areas of persistently high moisture that might relate to altered drainage from earthworks?]*

### 4.3. BSI (Bare Soil Index) - Example of a Soil Index

BSI = ((SWIR1 + Red) - (NIR + Blue)) / ((SWIR1 + Red) + (NIR + Blue))
This index requires SWIR1 (Band 11 for Sentinel-2). Let's check if we have it.

In [ ]:
try:
    SWIR1 = get_band(s2_data_xr, 'B11', 4) # Assuming B11 is the 5th band if not named
    swir1_float = SWIR1.astype(np.float32)
    # blue_float, red_float, nir_float already defined
    blue_float = BLUE.astype(np.float32)

    numerator = (swir1_float + red_float) - (nir_float + blue_float)
    denominator = (swir1_float + red_float) + (nir_float + blue_float)
    bsi = xr.where(denominator == 0, 0, numerator / denominator)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    plot_raster(bsi, ax, title='BSI (Bare Soil Index)', cmap='YlOrBr', cbar_label='BSI Value')
    plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_bsi.png")
    plt.show()
except ValueError as e:
    print(f"Could not calculate BSI: {e}. SWIR1 band (e.g., B11) might not be available in the processed file.")

**Observations (BSI):**
- *[TODO: Add observations if BSI was calculated. This index highlights bare soil areas. Are there any patterns of exposed soil that are not modern roads or fields? Could they relate to eroded earthworks or areas where specific soil types (perhaps managed by humans, like 'terra preta') are present near the surface?]*

### 4.4. Simple Ratio (NIR / Red) - Vegetation Vigor

This is a simpler index related to vegetation density/health.

In [ ]:
# nir_float, red_float already defined
simple_ratio = xr.where(red_float == 0, 0, nir_float / red_float)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
# Using a percentile clip for better visualization as ratios can have extreme values
if np.isnan(simple_ratio.data).any():
    vmin, vmax = np.nanpercentile(simple_ratio.data, [5, 95])
else:
    vmin, vmax = np.percentile(simple_ratio.data, [5, 95])

im = ax.imshow(simple_ratio, cmap='PiYG', vmin=vmin, vmax=vmax)
ax.set_title('Simple Ratio (NIR / Red)')
ax.set_xlabel("Easting (pixels)")
ax.set_ylabel("Northing (pixels)")
plt.colorbar(im, ax=ax, label='SR Value', fraction=0.046, pad=0.04)
ax.set_xticks([])
ax.set_yticks([])
plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_simple_ratio.png")
plt.show()

**Observations (Simple Ratio):**
- *[TODO: Add observations. Similar to NDVI, but more sensitive to high biomass. Note any unusual patterns of vegetation vigor or stress highlighted by this index.]*

## 5. Overlaying AOI Boundary (Contextual)

If an AOI geometry is available, overlay it on one of the visualizations for context.

In [ ]:
if aoi_geom is not None:
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    ax.imshow(true_color_composite) # Using true color as background
    
    # We need to plot the AOI in pixel coordinates or reproject the raster for geo-coordinates with geopandas. 
    # For simplicity here, if the AOI is in WGS84 and the raster is projected, direct overlay is complex.
    # This section would ideally use the raster's CRS and transform for accurate overlay with geopandas.
    # For now, this is a conceptual placeholder if CRS are not aligned for direct plotting.
    
    # Attempt to plot if AOI is GeoPandas object and raster has CRS
    if hasattr(aoi_geom, 'crs') and hasattr(s2_data_xr, 'rio'):
        try:
            # Ensure AOI is in the same CRS as the raster
            if aoi_geom.crs and s2_data_xr.rio.crs:
                if aoi_geom.crs.to_string().lower() != s2_data_xr.rio.crs.to_string().lower():
                    print(f"Reprojecting AOI from {aoi_geom.crs} to {s2_data_xr.rio.crs} for overlay.")
                    aoi_geom_reprojected = aoi_geom.to_crs(s2_data_xr.rio.crs)
                else:
                    aoi_geom_reprojected = aoi_geom
                
                # Geopandas plot needs the Axes object (ax) and the transform from rasterio
                # This part is tricky because imshow sets pixel extent, geopandas plots in geo-coords.
                # A better way is to use rasterio.plot.show with geopandas data.
                # Resetting plot for proper overlay with rasterio.plot.show
                plt.close(fig) # Close the previous imshow figure
                fig, ax = plt.subplots(1, 1, figsize=(12, 12))
                show(s2_data_xr.isel(band=[configured_bands.index(b) for b in ['B04', 'B03', 'B02']]), ax=ax, transform=s2_data_xr.rio.transform(), rgb_composite=True) # Example for True Color
                aoi_geom_reprojected.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2, label='AOI')
                ax.set_title('True Color Composite with AOI Overlay')
                plt.legend()
                plt.savefig(EDA_OUTPUT_DIR / f"{Path(selected_s2_path).stem}_true_color_with_aoi.png")
                plt.show()
            else:
                print("AOI or Raster CRS is undefined. Skipping overlay.")
        except Exception as e:
            print(f"Error during AOI overlay: {e}. Skipping overlay.")
            # Fallback to just showing the image if overlay fails
            plt.close(fig) # Close any partial figure
            fig, ax = plt.subplots(1, 1, figsize=(10,10))
            ax.imshow(true_color_composite)
            ax.set_title('True Color Composite (AOI Overlay Failed)')
            plt.show()
else:
    print("AOI geometry not available, skipping AOI overlay.")

## 6. Summary of Observations & Potential Anomalies

Based on the visualizations above:

1.  **Overall Landscape:**
    *   *[TODO: Briefly describe the general landscape characteristics of the selected AOI based on true/false color images and indices. E.g., predominantly dense forest, areas of agriculture, river systems, etc.]*

2.  **Potential Anomalies Noted:**
    *   **Feature 1 (Approx. Location/Pixel Coords, Description, Best seen in which composite/index?):**
        *   *e.g., A rectangular area of lower NDVI (approx. 200x300 pixels) surrounded by high NDVI forest, not corresponding to any visible modern clearing in true color. Visible near center of NDVI map.*
    *   **Feature 2 (Location, Description, Visualization):**
        *   *e.g., A faint linear feature visible in the False Color (NIR-R-G) composite, appearing as a slightly different shade of red, suggesting a subtle change in vegetation type or health. Runs E-W in the northern third.*
    *   **Feature 3 (Location, Description, Visualization):**
        *   *e.g., An area of mottled BSI values (if calculated) that doesn't align with current agricultural fields, suggesting varied soil composition. Best seen in BSI map, western edge.*

3.  **Interpretation Difficulty:**
    *   *[TODO: Note any challenges. E.g., Cloud shadows or haze remnants affecting index calculations? Difficulty distinguishing natural vegetation variations from potentially anthropogenic ones? Resolution limitations?]*

4.  **Next Steps for these Anomalies:**
    *   Examine these specific locations at higher resolution if possible (e.g., using commercial imagery if available, or by zooming in if current resolution allows more detail).
    *   Cross-reference with LiDAR data if available for the same area to see if there are corresponding topographic features.
    *   Search textual data for mentions of activity or features in these specific regions.
    *   Consider these for targeted feature engineering (e.g., texture analysis, object-based image analysis if applicable).